In [ ]:
import requests
import pandas as pd
import numpy as np
#import eurostat
import json
import os
import datetime as dt

In [ ]:
import os
print("Working directory:", os.getcwd())

In [ ]:
os.chdir('\\Users\\lorenz\\Library\\CloudStorage\\OneDrive-UniversitàdegliStudidiMilano\\PhD\\Progetto\\GaR\\src')
path = 'data'

In [ ]:
# IMPORT INDUSTRIAL PRODUCTION (IP) DATA 

ip = pd.read_csv("ip_by_sector.csv")

ip = ip[['nace_r2', 'geo', 'TIME_PERIOD', 'OBS_VALUE', 'unit']]

ip['Time'] = ip['TIME_PERIOD']

#ip['Time'] = pd.to_datetime(ip['Time'], format="%Y-%m")    

#ip['Time'] = ip['Time'].dt.strftime('%Y/%m')

ip.set_index('Time', inplace=True)

In [ ]:
# IMPORT GAS PRICES DATA 

gasp = pd.read_csv("gas_data.csv")

gasp[['Prices']] = gasp[['LSG Natural Gas TTF NL 1st Fut. Day - SETT. PRICE']]

gasp = gasp[['Time','Prices']]

gasp['Time'] = pd.to_datetime(gasp['Time'], format="%m/%d/%y")

month_gasp = gasp.groupby(gasp['Time'].dt.to_period('M')).mean() # group data and make monthly average to match IP data

month_gasp = month_gasp.drop(columns=['Time'])

month_gasp.loc['2004-01':'2023-12']

In [ ]:
df = ip.drop(columns=['TIME_PERIOD'])

df['geo'] = df['geo'].astype('category')

df['nace_r2'] = df['nace_r2'].astype('category')

df['unit'] = df['unit'].astype('category')

df_ue = df[df['geo'] == 'EU27_2020']

df_ue_c = df_ue[df_ue['nace_r2'] == 'C']

df_ue_c21 = df_ue_c[df_ue_c['unit'] == 'I15']

df_ue_c21['GAS_PRICE'] = month_gasp['Prices']